前準備

In [1]:
#numpy, matplotlib, pandasのインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import unicodedata

#日付データの変更のためインポート
import datetime as dt

#lightGBMのインポート
import lightgbm as lgb

#訓練データとテストデータ分割のためにインポート
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

#標準化のためインポート
from sklearn.preprocessing import StandardScaler

#エンコーディングのためインポート
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#精度検証のためインポート
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve

#ハイパーパラメータチューニングのためにインポート
from sklearn.model_selection import GridSearchCV

#特徴量選択のためにインポート
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV

#ロジスティック回帰のためにインポート
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

#SVMのためにインポート
from sklearn import svm

#決定木のためにインポート
from sklearn.tree import DecisionTreeClassifier, export_graphviz

#ランダムフォレストのためにインポート
from sklearn.ensemble import RandomForestClassifier

#アダブーストのためにインポート
from sklearn.ensemble import AdaBoostClassifier

#CatBoostのためにインポート
from catboost import CatBoostClassifier
from catboost import Pool

# SMOTEのためにインポート
# from imblearn.over_sampling import SMOTE

C:\Users\yutak\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# notebookの設定

# データフレームの最大表示行数、列数を広げる
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)

# 図のサイズ
plt.rcParams["figure.figsize"]=(8,4)

# matplotlibのフォント
font = 'Yu Mincho'

In [4]:
#学習データ、テストデータの読み込み
train0 = pd.read_csv('dataset/train.csv')
test0 = pd.read_csv('dataset/test.csv')

In [5]:
train0

id   Age    TypeofContact  CityTier DurationOfPitch      Occupation  \
0        0   50歳     Self Enquiry         2            900秒  Large Business   
1        1   56歳  Company Invited         1             14分        Salaried   
2        2   NaN     Self Enquiry         1             10分  Large Business   
3        3  三十七歳     Self Enquiry         2           1080秒  Small Business   
4        4   48歳  Company Invited         3           1020秒  Small Business   
...    ...   ...              ...       ...             ...             ...   
3484  3484   40歳     Self Enquiry         2           1560秒        Salaried   
3485  3485   40代     Self Enquiry         1              9分  Large Business   
3486  3486   31歳     Self Enquiry         1            840秒  Small Business   
3487  3487   56歳  Company Invited         2            900秒        Salaried   
3488  3488   42歳     Self Enquiry         1              9分  Small Business   

      Gender  NumberOfPersonVisiting  NumberOfFollowups ProductPitched  \
0       male                     1.0                4.0          Basic   
1       Male                     1.0                4.0       Standard   
2     Female                     1.0                3.0          Basic   
3     female                     1.0                3.0       Standard   
4     female                     1.0                3.0          Basic   
...      ...                     ...                ...            ...   
3484    Male                     2.0                3.0          Basic   
3485    Male                     3.0                3.0          Basic   
3486  FEMALE                     3.0                2.0       Standard   
3487    Male                     3.0                6.0           King   
3488    male                     3.0                1.0          Basic   

      PreferredPropertyStar NumberOfTrips  Passport  PitchSatisfactionScore  \
0                       3.0             5         1                       4   
1                       3.0             2         1                       4   
2                       3.0             4         0                       4   
3                       4.0             1         0                       5   
4                       4.0             4         0                       4   
...                     ...           ...       ...                     ...   
3484                    3.0             3         0                       1   
3485                    5.0             5         0                       3   
3486                    3.0             5         0                       4   
3487                    3.0             7         1                       4   
3488                    3.0             3         0                       1   

         Designation MonthlyIncome     customer_info  ProdTaken  
0          Executive      253905.0      未婚 車未所持 子供なし          1  
1     Senior Manager      404475.0     離婚済み,車あり,子供無し          0  
2          Executive      278145.0  結婚済み、自動車未所有,子供なし          1  
3     Senior Manager      326805.0     離婚済み、車所持、子供無し          0  
4          Executive      258435.0         独身／車所持／無子          1  
...              ...           ...               ...        ...  
3484       Executive      258900.0   離婚済み 乗用車所持 子供なし          1  
3485       Executive      260415.0     結婚済み、車所持、子供2人          0  
3486  Senior Manager      317340.0      独身／車所持／こども1人          0  
3487              VP      527910.0  結婚済み、自動車未所有、子供2人          1  
3488       Executive      278190.0    結婚済み,車未所持、子供1人          0  

[3489 rows x 18 columns]

In [6]:
test0

id  Age    TypeofContact  CityTier DurationOfPitch      Occupation  \
0     3489  ４８歳     Self Enquiry         2             13分  Small Business   
1     3490  30代     Self Enquiry         2             12分  Small Business   
2     3491  25歳     Self Enquiry         1            540秒        Salaried   
3     3492  21歳  Company Invited         2            420秒        Salaried   
4     3493  41歳  Company Invited         1              7分        Salaried   
...    ...  ...              ...       ...             ...             ...   
3484  6973  41歳  Company Invited         1             14分  Small Business   
3485  6974  44歳  Company Invited         1             35分        Salaried   
3486  6975  24歳     Self Enquiry         2             21分  Small Business   
3487  6976  25歳     Self Enquiry         1              9分  Small Business   
3488  6977  44才     Self Enquiry         1            480秒  Small Business   

      Gender  NumberOfPersonVisiting  NumberOfFollowups ProductPitched  \
0       Male                     1.0                4.0   Super De|uxe   
1     Ｆｅｍａｌｅ                     1.0                4.0       Standard   
2     Female                     1.0                4.0          Basic   
3       Male                     1.0                4.0          Basic   
4       MALE                     1.0                4.0          Basic   
...      ...                     ...                ...            ...   
3484  Female                     1.0                3.0          Basic   
3485    Male                     3.0                5.0         Deluxe   
3486    male                     2.0                3.0          basic   
3487    MALE                     2.0                3.0          Basic   
3488  female                     4.0                4.0         Deluxe   

      PreferredPropertyStar NumberOfTrips  Passport  PitchSatisfactionScore  \
0                       3.0             7         0                       3   
1                       3.0             4         1                       3   
2                       3.0             1         0                       3   
3                       4.0             1         0                       3   
4                       3.0             1         0                       4   
...                     ...           ...       ...                     ...   
3484                    3.0             2         0                       4   
3485                    3.0             3         0                       3   
3486                    3.0             2         0                       3   
3487                    3.0             2         0                       3   
3488                    3.0             5         0                       1   

         Designation MonthlyIncome     customer_info  
0                AVP      496950.0     結婚済み 車所持 子供なし  
1     Senior Manager      月収30.0万円    結婚済み、車未所持、子供なし  
2          Executive      月収26.0万円  離婚済み、自動車未所有、子供なし  
3     Senior Manager      259875.0   離婚済み、自動車所有、子供なし  
4          Executive      268830.0       独身/車所持／子供なし  
...              ...           ...               ...  
3484       Executive      261840.0     結婚済み/車なし／子供無し  
3485         Manager      349770.0  結婚済み、自家用車あり、子供1人  
3486       Executive      月収27.0万円      独身、車未所持、子供なし  
3487       Executive      272430.0   結婚済み、車未所持、こども1人  
3488         Manager      411795.0       未婚　車なし　子供3人  

[3489 rows x 17 columns]

In [7]:
# 不要な変数の削除

# 変数名と列番号の対応を表示
col = pd.DataFrame(train0.columns.values).T

# dropの実行,事故率に関係ないデータは予測に利用できないのでdrop(後々他の変数をdropしたくなった時楽な様に列番号でdropする)
train1 = train0.drop(train0.columns[0],axis=1) 
test1 = test0.drop(test0.columns[0],axis=1)

In [8]:
train1.isnull().sum()

Age                       100
TypeofContact               6
CityTier                    0
DurationOfPitch           121
Occupation                  0
Gender                      0
NumberOfPersonVisiting      0
NumberOfFollowups          33
ProductPitched              0
PreferredPropertyStar       0
NumberOfTrips              22
Passport                    0
PitchSatisfactionScore      0
Designation                 0
MonthlyIncome              56
customer_info               0
ProdTaken                   0
dtype: int64

In [9]:
#目的変数を分離する

train_x = train1.drop(['ProdTaken'], axis=1)
train_y = train1['ProdTaken']
test_x = test1



In [10]:
# 変数名と列番号の対応を表示
col = pd.DataFrame(train1.columns.values).T
col

0              1         2                3           4       5   \
0  Age  TypeofContact  CityTier  DurationOfPitch  Occupation  Gender   

                       6                  7               8   \
0  NumberOfPersonVisiting  NumberOfFollowups  ProductPitched   

                      9              10        11                      12  \
0  PreferredPropertyStar  NumberOfTrips  Passport  PitchSatisfactionScore   

            13             14             15         16  
0  Designation  MonthlyIncome  customer_info  ProdTaken

In [11]:
def preprocessing(train_x, train_y):
    # Age numeric
    # 漢数字とアラビア数字のマッピング
    kanji_to_num = {'一': 1, '二': 2, '三': 3, '四': 4, '五': 5, '六': 6, '七': 7, '八': 8, '九': 9,'十': 10, '百': 100, '千': 1000, '万': 10000,'零': 0, '〇': 0}
    def kanji_to_arabic(kanji):
        result = 0
        temp = 0
        for char in kanji:
            value = kanji_to_num.get(char, None)
            if value is not None:
                if value < 10:
                    if temp == 0:
                        temp = value
                    else:
                        temp = temp * 10 + value
                elif value >= 10:
                    if temp == 0:
                        temp = 1
                    result += temp * value
                    temp = 0
        return result + temp
    def process_age(age):
        if age is None or str(age) == 'nan':
            return None
        age = unicodedata.normalize('NFKC', age)
        age = ''.join([c for c in age if c.isdigit() or c in kanji_to_num])
        if age.isdigit():
            return int(age)
        return kanji_to_arabic(age)

    # TypeofContact categorical(dummy)
    def TypeofContact_to_dummy(str):
        if str == 'Self Enquiry':
            return 1
        elif str == 'Company Invited':
            return 0
        
    # CityTier 順序尺度
    
    # DurationOfPitch numeric
    def convert_to_minutes(duration):
        # durationがfloat型またはNoneである可能性があるため、文字列であることを確認
        if pd.isnull(duration):
            return None  # NaNの場合、Noneを返す
        duration = str(duration)  # 文字列に変換してエラーを防ぐ
        if '分' in duration:
            return float(duration.replace('分', ''))
        elif '秒' in duration:
            return float(duration.replace('秒', '')) / 60  # 秒を分に変換し、整数で返す

    # Occupation categorical
    def Occupation_to_dummy(str):
        if str == 'Large Business':
            return 2
        elif str == 'Small Business':
            return 1
        elif str == 'Salaried':
            return 0
       
    # Gender categorical
    def Gender_dealing(gender):
        # 文字列を半角に変換し、大文字に統一
        gender = unicodedata.normalize('NFKC', gender).upper().strip()
        # 不要な空白を削除
        gender = ''.join(gender.split())

        if 'FEMALE' in gender:
            return 1
        elif 'MALE' in gender:
            return 0
        else:
            return None  # 性別が識別できない場合はNoneを返す
        
    # NumberOfPersonVisiting numeric
    
    # NumberOfFollowups numeric
    def NumberOfFollowups_dealing(input_int):
        if input_int >= 100:
            return input_int /100
        else:
            return input_int
    
    # ProductPitched categorical
    # Designation categorical
    def standardize_str(input_str):
        # 文字列を半角に変換し、小文字に統一
        input_str = unicodedata.normalize('NFKC', input_str).lower().strip()
        # 不要な空白や特殊記号を削除
        input_str = ''.join(input_str.split())
        input_str = input_str.replace('|', 'l').replace('×', 'x').replace('𝘤', 'c').replace('𝖺', 'a').replace('𝙳', 'd')
        # その他特殊文字を通常の英字に置換
        input_str = input_str.replace('ᗞ', 'd').replace('𐊡', 'a').replace('𝘳', 'r').replace('ꓢ', 's').replace('ı', 'i')
        input_str = input_str.replace('β', 'b').replace('в', 'b').replace('с', 'c').replace('տ', 's').replace('ς', 'c')
        input_str = input_str.replace('ꭰ', 'd').replace('ε', 'e').replace('ι', 'i').replace('α', 'a').replace('ո', 'n')
        input_str = input_str.replace('ѕ', 's').replace('μ', 'm').replace('е', 'e').replace('а', 'a').replace('ѵ', 'v')
        input_str = input_str.replace('aasic', 'basic')
        return input_str
    
    # PreferredPropertyStar 順序尺度
    
    # NumberOfTrips numeric
    def NumberOfTrips_dealing(str):
        if pd.isnull(str):
            return None 
        if '半年に' in str:
            return 2 * int(str.replace('半年に', '').replace('回', ''))
        elif '年に' in str:
            return int(str.replace('年に', '').replace('回', ''))
        elif '四半期に' in str:
            return 4 * int(str.replace('四半期に', '').replace('回', ''))
        else :
            return int(str)
        
    # Passport categorical(dummy)
    
    # PitchSatisfactionScore 順序尺度だけど間隔尺度的要素あり
    
    # MonthlyIncome numeric
    def MonthlyIncome_dealing(input_str):
        if pd.isnull(input_str):
            return None 
        if '月収' in input_str:
            return 10000 * float(input_str.replace('月収', '').replace('万円', ''))
        elif '万円' in input_str:
            return 10000 * float(input_str.replace('万円', ''))
        else:
            return float(input_str)
        
    # customer_info
    def customer_info_dealing(input_str):
        # 文字列を半角に変換し、小文字に統一
        input_str = unicodedata.normalize('NFKC', input_str).lower().strip()
        # 不要な空白や特殊記号を削除
        input_str = input_str.replace('/', ' ').replace('／', ' ').replace('、', ' ').replace('　', ' ')
        input_str = input_str.replace('\u3000', ' ').replace('\t', ' ').replace('\n', ' ')
        input_str = re.sub(r'(?<=\S)\s+(?=\S)', ',', input_str, count=2)
        return input_str
    
    # married categorical
    
    # car_possesion categorival(dummy)
    def car_possesion_dealing(input_str):
        if input_str in ['車未所持', '自動車未所有', '自家用車なし', '乗用車なし', '車なし', '車保有なし', 0]:
            return 0
        elif input_str in ['車所持', '自動車所有', '自家用車あり', '乗用車所持', '車保有', '車あり', 1]:
            return 1
        
    # offspring -1以外はnumeric
    def offspring_dealing(input_str):
        if '1' in input_str:
            return 1
        elif '2' in input_str:
            return 2
        elif '3' in input_str:
            return 3
        else:
            return 0
    
    def offspring_identified_dealing(input_str):
        if input_str in ['子供の数不明', '不明', 'わからない', '子育て状況不明', '子の数不詳']:
            return 1
        else:
            return 0
        
    def dealing_missing_values(input_df):
        df = input_df.copy()
        df['Age'] = df['Age'].fillna(df['Age'].mean())
        df['TypeofContact'] = df['TypeofContact'].fillna(df['TypeofContact'].median())
        df['DurationOfPitch'] = df['DurationOfPitch'].fillna(df['DurationOfPitch'].mean())
        df['NumberOfFollowups'] = df['NumberOfFollowups'].fillna(df['NumberOfFollowups'].mean())
        df['NumberOfTrips'] = df['NumberOfTrips'].fillna(df['NumberOfTrips'].mean())
        df['MonthlyIncome'] = df['MonthlyIncome'].fillna(df['MonthlyIncome'].mean())
        return df
    
    def dummy_ex(feature, train_x, test_x):
        # OneHotEncoder の初期化時に sparse_output 引数を使用
        ohe = OneHotEncoder(sparse_output=False, categories='auto')
        new_array = pd.concat([train_x[[feature]], test_x[[feature]]], axis=0)
        ohe.fit(new_array)

        # ダミー変数の列名の作成
        columns = [f'{feature}_{v}' for v in ohe.categories_[0]]

        # 生成されたダミー変数をデータフレームに変換
        dummy_vals_train = pd.DataFrame(ohe.transform(train_x[[feature]]), columns=columns)
        dummy_vals_test = pd.DataFrame(ohe.transform(test_x[[feature]]), columns=columns)

        # 残りの変数と結合
        tr = pd.concat([train_x.drop([feature], axis=1), dummy_vals_train.reset_index(drop=True)], axis=1)
        te = pd.concat([test_x.drop([feature], axis=1), dummy_vals_test.reset_index(drop=True)], axis=1)

        return tr, te
    
    def function_apply(input_df):
        df = input_df.copy()
        df['Age'] = df['Age'].apply(process_age)
        df['TypeofContact'] = df['TypeofContact'].apply(TypeofContact_to_dummy)
        df['DurationOfPitch'] = df['DurationOfPitch'].apply(convert_to_minutes)
        df['Occupation'] = df['Occupation'].apply(Occupation_to_dummy)
        df['Gender'] = df['Gender'].apply(Gender_dealing)
        df['NumberOfFollowups'] = df['NumberOfFollowups'].apply(NumberOfFollowups_dealing)
        df['ProductPitched'] = df['ProductPitched'].apply(standardize_str)
        df['NumberOfTrips'] = df['NumberOfTrips'].apply(NumberOfTrips_dealing)
        df['Designation'] = df['Designation'].apply(standardize_str)
        df['MonthlyIncome'] = df['MonthlyIncome'].apply(MonthlyIncome_dealing)
        df['customer_info'] = df['customer_info'].apply(customer_info_dealing)
        df[['married', 'car_possesion', 'offspring']] = df['customer_info'].str.split(',', n=2, expand=True)
        df = df.drop(['customer_info'],axis=1)
        df['car_possesion'] = df['car_possesion'].apply(car_possesion_dealing)
        df['offspring'] = df['offspring'].apply(offspring_dealing)
        df['offspring_identified'] = df['offspring'].apply(offspring_identified_dealing)
        df = dealing_missing_values(df)
        return df
    
    dummy_col = ['CityTier', 'Occupation', 'ProductPitched', 'PreferredPropertyStar', 'PitchSatisfactionScore', 'Designation', 'married']
    std_feature = ['Age', 'DurationOfPitch', 'NumberOfPersonVisiting', 'NumberOfFollowups', 'NumberOfTrips', 'MonthlyIncome']
    
    def function_apply_both(train_x, test_x, std_feature):
        tr_x = train_x.copy()
        te_x = test_x.copy()
        tr_x = function_apply(tr_x)
        te_x = function_apply(te_x)
        for feature in dummy_col:
            tr_x, te_x = dummy_ex(feature, tr_x, te_x)
        std_sc = StandardScaler()
        tr_x[std_feature] = std_sc.fit_transform(tr_x[std_feature])
        te_x[std_feature] = std_sc.fit_transform(te_x[std_feature])
        return tr_x, te_x
    
    tr_x, te_x = function_apply_both(train_x, test_x, std_feature)
    
    print('Preprocessing finished')
    return tr_x, te_x

train_x, test_x = preprocessing(train_x, train_y)

Preprocessing finished


In [12]:
train_x.info()
train_x.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3489 entries, 0 to 3488
Data columns (total 40 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Age                         3489 non-null   float64
 1   TypeofContact               3489 non-null   float64
 2   DurationOfPitch             3489 non-null   float64
 3   Gender                      3489 non-null   int64  
 4   NumberOfPersonVisiting      3489 non-null   float64
 5   NumberOfFollowups           3489 non-null   float64
 6   NumberOfTrips               3489 non-null   float64
 7   Passport                    3489 non-null   int64  
 8   MonthlyIncome               3489 non-null   float64
 9   car_possesion               3489 non-null   int64  
 10  offspring                   3489 non-null   int64  
 11  offspring_identified        3489 non-null   int64  
 12  CityTier_1                  3489 non-null   float64
 13  CityTier_2                  3489 

Age                           0
TypeofContact                 0
DurationOfPitch               0
Gender                        0
NumberOfPersonVisiting        0
NumberOfFollowups             0
NumberOfTrips                 0
Passport                      0
MonthlyIncome                 0
car_possesion                 0
offspring                     0
offspring_identified          0
CityTier_1                    0
CityTier_2                    0
CityTier_3                    0
Occupation_0                  0
Occupation_1                  0
Occupation_2                  0
ProductPitched_basic          0
ProductPitched_deluxe         0
ProductPitched_king           0
ProductPitched_standard       0
ProductPitched_superdeluxe    0
PreferredPropertyStar_3.0     0
PreferredPropertyStar_4.0     0
PreferredPropertyStar_5.0     0
PitchSatisfactionScore_1      0
PitchSatisfactionScore_2      0
PitchSatisfactionScore_3      0
PitchSatisfactionScore_4      0
PitchSatisfactionScore_5      0
Designat

In [13]:
train_x

Age  TypeofContact  DurationOfPitch  Gender  \
0     1.129079e+00            1.0         0.095918       0   
1     1.738839e+00            0.0        -0.050458       0   
2    -7.221012e-16            1.0        -0.635964       1   
3    -1.920682e-01            1.0         0.535048       1   
4     9.258256e-01            0.0         0.388672       1   
...            ...            ...              ...     ...   
3484  1.128119e-01            1.0         1.706061       0   
3485  1.128119e-01            1.0        -0.782341       0   
3486 -8.018284e-01            1.0        -0.050458       1   
3487  1.738839e+00            0.0         0.095918       0   
3488  3.160653e-01            1.0        -0.782341       0   

      NumberOfPersonVisiting  NumberOfFollowups  NumberOfTrips  Passport  \
0                  -1.919689           0.374125       1.045266         1   
1                  -1.919689           0.374125      -0.691722         1   
2                  -1.919689          -0.756099       0.466270         0   
3                  -1.919689          -0.756099      -1.270718         0   
4                  -1.919689          -0.756099       0.466270         0   
...                      ...                ...            ...       ...   
3484               -0.543241          -0.756099      -0.112726         0   
3485                0.833207          -0.756099       1.045266         0   
3486                0.833207          -1.886323       1.045266         0   
3487                0.833207           2.634574       2.203258         1   
3488                0.833207          -3.016547      -0.112726         0   

      MonthlyIncome  car_possesion  offspring  offspring_identified  \
0         -1.364953              0          0                     0   
1          0.654584              1          0                     0   
2         -1.039831              0          0                     0   
3         -0.387174              1          0                     0   
4         -1.304194              1          0                     0   
...             ...            ...        ...                   ...   
3484      -1.297957              1          0                     0   
3485      -1.277637              1          2                     0   
3486      -0.514124              1          1                     0   
3487       2.310171              0          2                     0   
3488      -1.039228              0          1                     0   

      CityTier_1  CityTier_2  CityTier_3  Occupation_0  Occupation_1  \
0            0.0         1.0         0.0           0.0           0.0   
1            1.0         0.0         0.0           1.0           0.0   
2            1.0         0.0         0.0           0.0           0.0   
3            0.0         1.0         0.0           0.0           1.0   
4            0.0         0.0         1.0           0.0           1.0   
...          ...         ...         ...           ...           ...   
3484         0.0         1.0         0.0           1.0           0.0   
3485         1.0         0.0         0.0           0.0           0.0   
3486         1.0         0.0         0.0           0.0           1.0   
3487         0.0         1.0         0.0           1.0           0.0   
3488         1.0         0.0         0.0           0.0           1.0   

      Occupation_2  ProductPitched_basic  ProductPitched_deluxe  \
0              1.0                   1.0                    0.0   
1              0.0                   0.0                    0.0   
2              1.0                   1.0                    0.0   
3              0.0                   0.0                    0.0   
4              0.0                   1.0                    0.0   
...            ...                   ...                    ...   
3484           0.0                   1.0                    0.0   
3485           1.0                   1.0                    0.0   
3486           0.0                  

In [14]:
Train_x = train_x.copy()
Test_x = test_x.copy()


# 各モデルの比較

## ①lightGBM

In [15]:
#LightGBMのハイパーパラメータ設定
params = {
    'objective': 'binary', 
    'metric': 'auc',
    'num_leaves': 23,
    'learning_rate': 0.05, 
    'feature_fraction': 0.4, 
    'bagging_fraction': 0.8,
    'bagging_freq': 3,
    'verbose': -1,
    'lambda_l1': 0.81,
    'lambda_l2': 90
}
auc_sum = []
preds = []
splits = 7
kf2 = KFold(n_splits = splits, shuffle = True, random_state = 53)
for fold, (train_idx, valid_idx) in enumerate(kf2.split(Train_x)):
    X_tr, y_tr = Train_x.iloc[train_idx, :], train_y.iloc[train_idx]
    X_val, y_val = Train_x.iloc[valid_idx, :], train_y.iloc[valid_idx]
    
    train_set = lgb.Dataset(X_tr, y_tr)
    val_set = lgb.Dataset(X_val, y_val)
    
    model = lgb.train(params, train_set, num_boost_round = 500, early_stopping_rounds = 100, valid_sets = [train_set, val_set], verbose_eval = 100)
    
    y_pred = model.predict(X_val)
    auc = roc_auc_score(y_val, y_pred)
    pred = model.predict(Test_x)
    preds.append(pred)
    
    print(f'Fold {fold + 1}: AUC = {auc:4f}')
    auc_sum.append(auc)

#特徴量の重要度を出力する
importances = model.feature_importance()
column_names = Train_x.columns.values
temp = []
for i in range(len(column_names)):
    temp.append([importances[i],column_names[i]])
temp.sort(reverse=True, key=lambda x:x[0])
for i in range(len(temp)):
    print(temp[i][0]," : ",temp[i][1])
    
print(np.mean(auc_sum))

last_pred_1 = 3489 * [0.0]
for i in range(3489):
    for j in range(splits):
        last_pred_1[i] += preds[j][i]
    last_pred_1[i] /= splits

ensemble_preds = []
ensemble_preds.append(last_pred_1)

Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.851171	valid_1's auc: 0.84692
[200]	training's auc: 0.869103	valid_1's auc: 0.850101
[300]	training's auc: 0.880692	valid_1's auc: 0.850812
Early stopping, best iteration is:
[244]	training's auc: 0.874968	valid_1's auc: 0.85216
Fold 1: AUC = 0.852160
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.848117	valid_1's auc: 0.865083
[200]	training's auc: 0.867302	valid_1's auc: 0.865858
Early stopping, best iteration is:
[157]	training's auc: 0.86084	valid_1's auc: 0.867476
Fold 2: AUC = 0.867476
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.850226	valid_1's auc: 0.845187
[200]	training's auc: 0.870122	valid_1's auc: 0.853951
Early stopping, best iteration is:
[151]	training's auc: 0.863077	valid_1's auc: 0.856034
Fold 3: AUC = 0.856034
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.853621	

In [ ]:
#提出用ファイルの作成
submission = pd.DataFrame({'ID':test0['id'], 'ProdTaken':last_pred_1})
submission.to_csv('YutaK2.csv', index=False, header=None)

## ②LogisticRegression(アンサンブルで0倍が最適のため不採用)

In [16]:
auc_sum = []
preds = []
splits = 7
kf2 = KFold(n_splits = splits, shuffle = True, random_state = 53)
for fold, (train_idx, valid_idx) in enumerate(kf2.split(Train_x)):
    X_tr, y_tr = Train_x.iloc[train_idx, :], train_y.iloc[train_idx]
    X_val, y_val = Train_x.iloc[valid_idx, :], train_y.iloc[valid_idx]
    
    model = LogisticRegression(penalty = 'none',
                               dual = False,
                               tol = 0.001,
                               C = 1.0,
                               fit_intercept = True,
                               intercept_scaling = 1,
                               class_weight = None,
                               random_state = None,
                               solver = 'saga',
                               max_iter = 100,
                               multi_class = 'auto',
                               verbose = 0,
                               warm_start = False,
                               n_jobs = None,
                               l1_ratio = None
                              )
    #訓練データをモデルに適合させる
    model.fit(X_tr, y_tr)
    
    #訓練データから予測 & aucの算出
    y_pred_2d = model.predict_proba(X_val)
    y_pred = y_pred_2d[:,1]
    auc = roc_auc_score(y_val, y_pred)
    
    pred_2d = model.predict_proba(Test_x)
    pred = pred_2d[:,1]
    preds.append(pred)
    test_size = len(pred)
    
    print(f'Fold {fold + 1}: AUC = {auc:.4f}')
    auc_sum.append(auc)

print(np.mean(auc_sum))

last_pred_2 = 3489 * [0.0]
for i in range(3489):
    for j in range(splits):
        last_pred_2[i] += preds[j][i]
    last_pred_2[i] /= splits

InvalidParameterError: The 'penalty' parameter of LogisticRegression must be a str among {'l1', 'elasticnet', 'l2'} or None. Got 'none' instead.

## ③SGDClassifier(アンサンブルで0倍が最適のため不採用)

In [ ]:
auc_sum = []
preds = []
splits = 7
kf2 = KFold(n_splits=splits,shuffle=True,random_state=71)

for fold, (train_idx, valid_idx) in enumerate(kf2.split(Train_x)):
    X_tr, y_tr = Train_x.iloc[train_idx, :], train_y.iloc[train_idx]
    X_val,y_val = Train_x.iloc[valid_idx, :], train_y.iloc[valid_idx]
    
    #ロジスティック回帰のインスタンスを生成
    lr = SGDClassifier(penalty='elasticnet',
                       alpha = 0.003,
                       l1_ratio = 0.3,
                       tol=0.0001,
                       fit_intercept=True,
                       class_weight='balanced',
                       random_state=None,
                       max_iter=1000,
                       verbose=0,
                       warm_start=False,
                       n_jobs=None,
                       loss ='log_loss'
                       )
    #訓練データをモデルに適合させる
    lr.fit(X_tr,y_tr)
    
    #訓練データから予測 & aucの算出
    y_pred_2d = lr.predict_proba(X_val)
    y_pred = y_pred_2d[:,1]
    auc = roc_auc_score(y_val, y_pred)
    
    pred_2d = lr.predict_proba(Test_x)
    pred = pred_2d[:,1]
    preds.append(pred)
    test_size = len(pred)
    
    print(f'Fold {fold + 1}: AUC = {auc:.4f}')
    auc_sum.append(auc)

print(np.mean(auc_sum))

last_pred_3 = 3489 * [0.0]
for i in range(3489):
    for j in range(splits):
        last_pred_3[i] += preds[j][i]
    last_pred_3[i] /= splits

## ④SVM(処理に時間がかかりすぎたため不採用)

In [ ]:
'''
from sklearn.pipeline import Pipeline

auc_sum = []
preds = []
splits = 7
kf2 = KFold(n_splits = splits, shuffle = True, random_state = 53)
for fold, (train_idx, valid_idx) in enumerate(kf2.split(train_x)):
    X_tr, y_tr = Train_x.iloc[train_idx, :], train_y.iloc[train_idx]
    X_val, y_val = Train_x.iloc[valid_idx, :], train_y.iloc[valid_idx]
    
    clf = svm.SVC(kernel = 'linear', probability = True)
    
    #訓練データをモデルに適合させる
    clf.fit(X_tr,y_tr)
    
    #訓練データから予測 & aucの算出
    y_pred_2d = clf.predict(X_val, prediction_type='Probability')
    y_pred = y_pred_2d[:,1]
    auc = roc_auc_score(y_val, y_pred)
    
    pred_2d = clf.predict(Test_x, prediction_type='Probability')
    pred = pred_2d[:,1]
    preds.append(pred)
    test_size = len(pred)
    
    print(f'Fold {fold + 1}: AUC = {auc:.4f}')
    auc_sum.append(auc)

print(np.mean(auc_sum))

last_pred_4 = len(test_y) * [0.0]
for i in range(len(test_y)):
    for j in range(splits):
        last_pred_4[i] += preds[j][i]
    last_pred_4[i] /= splits


curve = roc_curve(test_y, last_pred_4)
plt.plot(curve[0], curve[1])
auc_test = roc_auc_score(test_y, last_pred_4)
print(auc_test)
'''

pass

## ⑤決定木(他に比べて明らかにスコアが低いので不採用)

In [ ]:
from sklearn import tree

auc_sum = []
preds = []
splits = 7
kf2 = KFold(n_splits = splits, shuffle = True, random_state = 53)
for fold, (train_idx, valid_idx) in enumerate(kf2.split(Train_x)):
    X_tr, y_tr = Train_x.iloc[train_idx, :], train_y.iloc[train_idx]
    X_val, y_val = Train_x.iloc[valid_idx, :], train_y.iloc[valid_idx]
    
    clf = tree.DecisionTreeClassifier(criterion = 'gini',
                                      splitter = 'best',
                                      max_depth = 4, 
                                      min_samples_split = 3,
                                      min_samples_leaf = 1,
                                      min_weight_fraction_leaf = 0.0,
                                      max_features = 4,
                                      random_state = None,
                                      max_leaf_nodes = 8,
                                      min_impurity_decrease = 1e-07,
                                      class_weight = 'balanced'
                                     )
    #訓練データをモデルに適合させる
    clf.fit(X_tr,y_tr)
    
    #訓練データから予測 & aucの算出
    y_pred_2d = clf.predict_proba(X_val)
    y_pred = y_pred_2d[:,1]
    auc = roc_auc_score(y_val, y_pred)
    
    pred_2d = clf.predict_proba(Test_x)
    pred = pred_2d[:,1]
    preds.append(pred)
    test_size = len(pred)
    
    print(f'Fold {fold + 1}: AUC = {auc:.4f}')
    auc_sum.append(auc)

print(np.mean(auc_sum))

last_pred_5 = 3489 * [0.0]
for i in range(3489):
    for j in range(splits):
        last_pred_5[i] += preds[j][i]
    last_pred_5[i] /= splits

## ⑥ランダムフォレスト

In [ ]:
auc_sum = []
preds = []
splits = 7
kf2 = KFold(n_splits = splits, shuffle = True, random_state = 53)
for fold, (train_idx, valid_idx) in enumerate(kf2.split(Train_x)):
    X_tr, y_tr = Train_x.iloc[train_idx, :], train_y.iloc[train_idx]
    X_val, y_val = Train_x.iloc[valid_idx, :], train_y.iloc[valid_idx]
    
    clf = RandomForestClassifier(random_state = 53)
    #訓練データをモデルに適合させる
    clf.fit(X_tr,y_tr)
    
    #訓練データから予測 & aucの算出
    y_pred_2d = clf.predict_proba(X_val)
    y_pred = y_pred_2d[:,1]
    auc = roc_auc_score(y_val, y_pred)
    
    pred_2d = clf.predict_proba(Test_x)
    pred = pred_2d[:,1]
    preds.append(pred)
    test_size = len(pred)
    
    print(f'Fold {fold + 1}: AUC = {auc:.4f}')
    auc_sum.append(auc)

print(np.mean(auc_sum))

last_pred_6 = 3489 * [0.0]
for i in range(3489):
    for j in range(splits):
        last_pred_6[i] += preds[j][i]
    last_pred_6[i] /= splits

# ensemble_preds.append(last_pred_6)

# ⑦アダブースト

In [ ]:
auc_sum = []
preds = []
splits = 7
kf2 = KFold(n_splits = splits, shuffle = True, random_state = 53)
for fold, (train_idx, valid_idx) in enumerate(kf2.split(Train_x)):
    X_tr, y_tr = Train_x.iloc[train_idx, :], train_y.iloc[train_idx]
    X_val, y_val = Train_x.iloc[valid_idx, :], train_y.iloc[valid_idx]
    
    clf = AdaBoostClassifier(random_state = 53)
    #訓練データをモデルに適合させる
    clf.fit(X_tr,y_tr)
    
    #訓練データから予測 & aucの算出
    y_pred_2d = clf.predict_proba(X_val)
    y_pred = y_pred_2d[:,1]
    auc = roc_auc_score(y_val, y_pred)
    
    pred_2d = clf.predict_proba(Test_x)
    pred = pred_2d[:,1]
    preds.append(pred)
    test_size = len(pred)
    
    print(f'Fold {fold + 1}: AUC = {auc:.4f}')
    auc_sum.append(auc)

print(np.mean(auc_sum))

last_pred_7 = 3489 * [0.0]
for i in range(3489):
    for j in range(splits):
        last_pred_7[i] += preds[j][i]
    last_pred_7[i] /= splits

# ensemble_preds.append(last_pred_7)

## ⑧catboost

In [ ]:
auc_sum = []
preds = []
splits = 7
kf2 = KFold(n_splits = splits, shuffle = True, random_state = 74)
for fold, (train_idx, valid_idx) in enumerate(kf2.split(Train_x)):
    X_tr, y_tr = Train_x.iloc[train_idx, :], train_y.iloc[train_idx]
    X_val, y_val = Train_x.iloc[valid_idx, :], train_y.iloc[valid_idx]
    
    train_pool = Pool(X_tr, label = y_tr)
    test_pool  = Pool(X_val,label = y_val)
    
    params = {
        'depth' : 6,
        'learning_rate' : 0.16,
        'early_stopping_rounds' : 10,
        'iterations' : 200,
        'custom_loss' : ['Accuracy'],
        'random_seed' : 42,   
    }

    clf = CatBoostClassifier(**params)
    #訓練データをモデルに適合させる
    clf.fit(train_pool, eval_set = test_pool)
    
    #訓練データから予測 & aucの算出
    y_pred_2d = clf.predict(X_val, prediction_type='Probability')
    y_pred = y_pred_2d[:,1]
    auc = roc_auc_score(y_val, y_pred)
    
    pred_2d = clf.predict(Test_x, prediction_type='Probability')
    pred = pred_2d[:,1]
    preds.append(pred)
    test_size = len(pred)
    
    print(f'Fold {fold + 1}: AUC = {auc:.4f}')
    auc_sum.append(auc)

print(np.mean(auc_sum))

last_pred_8 = 3489 * [0.0]
for i in range(3489):
    for j in range(splits):
        last_pred_8[i] += preds[j][i]
    last_pred_8[i] /= splits

ensemble_preds.append(last_pred_8)

# 最終スコアの算出

## アンサンブル

In [ ]:
from itertools import product

def find_permutations(n):
    perm_list = []
    current_perm = [0] * n
    
    while True:
        if sum(current_perm) == 10:
            perm_list.append(tuple(current_perm))
    
        i = n-1
        while i >= 0 and current_perm[i] == 10:
            current_perm[i] = 0
            i -= 1
            
        if i < 0:
            break
        
        current_perm[i] += 1
    
    return perm_list

n = 4
permutations = find_permutations(n)
last_pred = len(test_y) * [0.0]
l = []
for i in range(n):
    l.append(0.0)
best_score = 0
for perm in permutations:
    for i in range(len(test_y)):
        last_pred_list = len(test_y) * l
        last_pred_list[i] = [last_pred_1[i], last_pred_6[i], last_pred_7[i], last_pred_8[i]]
        last_pred[i] = np.dot(last_pred_list[i], perm) / 10
    auc_test = roc_auc_score(test_y, last_pred)
    if auc_test > best_score:
        best_score = auc_test
        best_perm = perm
    print(perm, auc_test)
print('best param is ', best_perm, ', best score is ', best_score)

In [ ]:
def max_score(preds,dif,test_y,over_value_sum = [0.0], depth = [0],max = [0.0], ensemble_pred = [0.0],max_ratio = [0.0], now_ratio = [0.0]):
    
    if len(preds) == depth[0]+1: #最深部での処理
        temp_ensemble_pred = len(preds[0])*[0.0]
        
        for i in range(len(preds[0])):
            temp_ensemble_pred[i] = ensemble_pred[i]
            
        value = 1.0 - over_value_sum[0]
        now_ratio[depth[0]] = value
        
        for i in range(len(preds[0])):
            ensemble_pred[i] += preds[depth[0]][i]*value
        ensemble_auc = roc_auc_score(test_y,ensemble_pred)
        
        for i in range(len(preds[0])):
            ensemble_pred[i] = temp_ensemble_pred[i]
            
        if ensemble_auc > max[0]:
            for i in range(len(now_ratio)):
                max_ratio[i] = now_ratio[i]
            max[0] = ensemble_auc
        
        
    elif depth[0] == 0:              #最浅部での処理
        ensemble_pred = len(preds[0])*[0.0] #最深部でAUCを計算するためのリスト
        max_ratio  = len(preds)*[0.0]       #AUCの最大値を更新した際に、それを与えたアンサンブル割合を格納するリスト
        now_ratio  = len(preds)*[0.0]       #現在のアンサンブル割合を格納するリスト
        while over_value_sum[0]<=1.0+dif:
                
            max_score(preds=preds,dif=dif,test_y=test_y,over_value_sum= over_value_sum,depth = [1], max = max, ensemble_pred = ensemble_pred, max_ratio = max_ratio, now_ratio = now_ratio)
            
            for i in range(len(preds[0])):
                ensemble_pred[i] += preds[depth[0]][i]*dif
                
            now_ratio[depth[0]] += dif
            over_value_sum[0] += dif
            
        return [max[0],max_ratio]
    else:  #その他の深さでの処理
        now_ratio[depth[0]] = 0.0
        temp_ratio = now_ratio
        temp_over_value_sum = over_value_sum[0]
        
        temp_ensemble_pred = len(preds[0])*[0.0]
        for i in range(len(preds[0])):
            temp_ensemble_pred[i] = ensemble_pred[i]
            
        while over_value_sum[0]<=1.0:
                
            max_score(preds=preds,dif=dif,test_y=test_y,over_value_sum = over_value_sum, depth = [depth[0]+1],max = max, ensemble_pred = ensemble_pred, max_ratio = max_ratio, now_ratio = now_ratio)
            
            for i in range(len(preds[0])):
                ensemble_pred[i] += preds[depth[0]][i]*dif
                
            now_ratio[depth[0]] += dif
            over_value_sum[0] += dif
        now_ratio = temp_ratio
        over_value_sum[0] = temp_over_value_sum
        
        for i in range(len(preds[0])):
            ensemble_pred[i] = temp_ensemble_pred[i]
            
        now_ratio[depth[0]] = 0.0
        
#アンサンブル処理
dif = 0.05
result = max_score(preds = ensemble_preds, dif = dif, test_y = test_y)
print(result)